In [1]:
import Collobrative_Filtering as CF

import pandas as pd
import numpy as np
import inspect
import os

In [2]:
file_path = inspect.getfile(inspect.currentframe())
file_direction = os.path.dirname(os.path.abspath(file_path))
offertagging = os.path.join(file_direction, 'MJ_OFFER.xlsx')
usertagging = os.path.join(file_direction, 'MJ_TAG.xlsx')

In [3]:
offertagging = pd.read_excel(offertagging)

In [4]:
usertagging = pd.read_excel(usertagging)
usertagging = usertagging.groupby(['ID','UTID']).size().reset_index(name='COUNT')
usertagging = usertagging.pivot_table(index=['ID'], columns='UTID',values ='COUNT').fillna(0)

In [5]:
model = CF.COLLOBORATIVE_FILTERING()

In [6]:
offertagging.head()

,ID,LABEL_ID,OFFER_ID
0,864,54,OFF0007
1,864,54,OFF0007
2,864,54,OFF0007
3,864,54,OFF0007
4,864,54,OFF0007


In [7]:
rating_table = model.fit(usertagging,offertagging)

Starting likes info
Number of users: 310
Number of models: 214
Sparsity: 4.702%
Ending likes info
Number of users: 310
Number of models: 214
Sparsity: 4.702%
offer_rating pickle Done!
user_sparse pickle Done!
offer_mapping pickle Done!
tag_mapping pickle Done!


In [8]:
import redis
import json

In [9]:
redis_conn = redis.Redis(host='localhost',port=6379,db=0)

In [10]:
test = model.predict(json.loads(redis_conn.get(1)))

In [11]:
offerdata = offertagging[['LABEL_ID','OFFER_ID']].drop_duplicates().reset_index(drop=True)

In [25]:
offerdata['IND'] = 1

In [28]:
offerdata = offerdata.pivot_table(index='OFFER_ID',columns='LABEL_ID',values='IND').fillna(0)

In [12]:
offer = offertagging.OFFER_ID.unique()

In [34]:
np.array(test)*214/250

array([116,  83, 201, 115, 123,  53,  92, 182, 184,  41,  88, 173,  47,
        46, 199])

In [39]:
tagging_table = np.zeros(214)
for i in np.array(test)*214/250:
    tagging_table[i] = 1

In [35]:
from sklearn.metrics import pairwise

In [41]:
pairwise.cosine_similarity(tagging_table.reshape(1,-1), offerdata.values)

array([[ 0.18257419,  0.23094011,  0.25819889,  0.25819889,  0.08164966,
         0.09128709,  0.19518001,  0.07784989,  0.0745356 ,  0.09128709,
         0.10540926,  0.        ,  0.09128709,  0.        ,  0.        ,
         0.        ,  0.11547005,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.29277002,  0.31622777,  0.27386128,
         0.        ,  0.        ,  0.07161149,  0.        ,  0.09128709,
         0.07784989,  0.        ,  0.09759001,  0.        ,  0.        ,
         0.09759001,  0.        ,  0.07161149,  0.07161149]])